# SPARQL proficiency

The process of calculating the SPARQL proficiency metric.

In [ ]:
import json
import time
import re
import requests
import pandas as pd

color_palette = ['#5565cf', '#b14ecf', '#cf4773', '#cf913f', '#87cf38', '#30cf6b', '#29a1cf', '#2138cf']

file_path = 'data_stories_summary.json'
with open(file_path, 'r') as file:
    data = json.load(file)

## Correct score

In [ ]:
# Define expected rules based on query type
expected_variables_by_type = {
    'table': None,  # No constraints on variable names, labels can be used
    'count': ['count'],  # Must have ?count of type number
    'barchart': ['label', 'count'],  # Option 1: ?label and ?count, Option 2: only ?label
    'linechart': ['label', 'count'],
    'doughnutchart': ['label', 'count'],
    'scatterplot': ['x', 'y'],  # Must have ?x and ?y
    'textsearch': None,  # No specific constraints, <<searchterm>> placeholder is normal
    'map': ['point', 'lat', 'long']  # Must have ?point, ?lat, and ?long
}

# Function to test the SPARQL query and validate based on type
def test_sparql_query(endpoint, query, default_search_term='"example_search_term"', max_retries=3, delay=30):
    headers = {
        'Accept': 'application/sparql-results+json'
    }

    # Replace the placeholder <<searchterm>> with the default search term
    query = query.replace("<<searchterm>>", default_search_term)

    params = {
        'query': query
    }
    
    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(endpoint, headers=headers, params=params)
            
            # Check for any 2xx status code (successful responses)
            if 200 <= response.status_code < 300:
                print(f"Query to {endpoint} succeeded with status code {response.status_code}.")
                
                # Attempt to parse the JSON response
                try:
                    response_json = response.json()  # Ensure the response is valid JSON
                    return True, response_json
                except json.JSONDecodeError:
                    print(f"Failed to parse JSON for {query}. Response content: {response.text}")
                    return True, None  # Consider it a successful query but with invalid JSON
                
            elif response.status_code == 429:
                print(f'Rate limit hit (429). Retrying in {delay} seconds...')
                time.sleep(delay)
                retries += 1
            else:
                print(f'{response.status_code} for {endpoint}, {query}')
                return False, None
        except Exception as e:
            print(f'Exception for {endpoint}, {query}: {e}')
            return False, None
    return False, None

# Function to check for query correctness based on result variables and type-specific rules
def check_query_correctness(response_json, query_type):
    # If the response_json is None, we cannot validate the correctness
    if response_json is None:
        print(f"No valid JSON to check for query type {query_type}.")
        return False
    
    # Handling each type of query based on specific rules
    if query_type == 'table':
        print("Checking table query")
        return True
    
    elif query_type == 'count':
        print("Checking count query")
        if 'head' in response_json and 'vars' in response_json['head']:
            returned_vars = response_json['head']['vars']
            if 'count' in returned_vars:
                # Now check if the count is a number
                if 'results' in response_json and 'bindings' in response_json['results']:
                    for binding in response_json['results']['bindings']:
                        if 'count' in binding and binding['count']['type'] == 'literal' and binding['count'].get('datatype') == 'http://www.w3.org/2001/XMLSchema#integer':
                            return True
                print(f"Query did not return a valid number for ?count")
        return False

    elif query_type in ['barchart', 'linechart', 'doughnutchart']:
        print(f"Checking {query_type} query")
        if 'head' in response_json and 'vars' in response_json['head']:
            returned_vars = response_json['head']['vars']
            if ('label' in returned_vars and 'count' in returned_vars) or ('label' in returned_vars):
                return True
        return False
    
    elif query_type == 'scatterplot':
        print("Checking scatterplot query")
        if 'head' in response_json and 'vars' in response_json['head']:
            returned_vars = response_json['head']['vars']
            if 'x' in returned_vars and 'y' in returned_vars:
                return True
        return False

    elif query_type == 'textsearch':
        print("Checking textsearch query")
        return True  # Assume correctness as the search term is dynamically replaced
    
    elif query_type == 'map':
        print("Checking map query")
        if 'head' in response_json and 'vars' in response_json['head']:
            returned_vars = response_json['head']['vars']
            if 'point' in returned_vars and 'lat' in returned_vars and 'long' in returned_vars:
                return True
        return False

    else:
        print(f"Unrecognized query type: {query_type}")
        return False

story_sparql_metrics = []

# Iterate over the stories in the JSON
for story_id, story in data.items():
    sparql_endpoint = story['sparql_endpoint']
    queries = story.get('queries', [])

    total_queries = 0
    working_queries = 0
    correct_queries = 0

    for query_dict in queries:
        query = query_dict.get('query', '')
        query_type = query_dict.get('type', '')
        total_queries += 1
        query = re.sub(r'(^|\s)#.*?(\r\n|\n)', r'\1', query)
        query = re.sub(r'\s+', ' ', query).strip()

        success, response_json = test_sparql_query(sparql_endpoint, query)
        
        if success:
            working_queries += 1
            # Check query correctness based on response variables and result content
            if response_json:
                if check_query_correctness(response_json, query_type):
                    correct_queries += 1
        # Delay of 5 seconds between queries to avoid hitting rate limits
        time.sleep(5)

    # Calculate the percentage of working queries
    if total_queries > 0:
        working_percentage = (working_queries / total_queries) * 100
        correct_percentage = (correct_queries / total_queries) * 100
    else:
        working_percentage = 0
        correct_percentage = 0
    
    # composite SPARQL proficiency score (normalized to [0, 1]
    # proficiency_score = (working_percentage + correct_percentage) / 200.0
    story_sparql_metrics.append({
        'story_id': story_id,
        'total_queries': total_queries,
        'working_queries': working_queries,
        'correct_queries': correct_queries,
        'working_percentage': working_percentage,
        'correct_percentage': correct_percentage,
        # 'sparql_proficiency': round(proficiency_score, 2)
    })

    print(f"Story ID: {story_id} | Total: {total_queries} | Working: {working_queries} "
          f"({working_percentage:.2f}%) | Correct: {correct_queries} ({correct_percentage:.2f}%)")

df_sparql = pd.DataFrame(story_sparql_metrics)
df_sparql.to_csv('outputs/sparql_queries_score.csv')

In [2]:
df_sparql_queries = pd.read_csv('outputs/sparql_queries_score.csv', index_col=0)
df_sparql_queries['combined_normalized'] = ((df_sparql_queries['working_percentage'] + df_sparql_queries['correct_percentage']) / 200.0).round(2)
df_sparql_queries['correct_normalized'] = (df_sparql_queries['correct_percentage'] / 100.0)

df_sparql_proficiency = df_sparql_queries[['story_id', 'combined_normalized', 'correct_normalized']].copy()
df_sparql_proficiency.to_csv('outputs/sparql_proficiency_stage_one.csv')

## Federated bonus

In [3]:
# Regular expression to capture all federated endpoints in SERVICE clauses
service_pattern = re.compile(r'SERVICE\s*<([^>]+)>', re.IGNORECASE)

# Initialize counters
federation_metrics = []
data_sources_list = []

# Loop through the stories in the JSON data (per story)
for story_id, story_data in data.items():
    queries = story_data.get('queries', [])
    
    main_endpoint = story_data.get("sparql_endpoint", "")
    if main_endpoint:
        data_sources_list.append(main_endpoint.split("//")[1])
    
    total_queries_story = 0
    federated_queries_story = 0
    
    # Loop through each query in the story
    for query_dict in queries:
        total_queries_story += 1
        query = query_dict.get('query', '')
        query = re.sub(r'(^|\s)#.*?(\r\n|\n)', r'\1', query)
        query = re.sub(r'\s+', ' ', query).strip()
        
        service_matches = service_pattern.findall(query)
        if service_matches:
            federated_queries_story += 1
    
    # Calculate federated bonus for this story (fraction of federated queries)
    federated_bonus = (federated_queries_story / total_queries_story) if total_queries_story > 0 else 0
    
    # Store per-story federation metrics (using story_id as key)
    federation_metrics.append({
        'story_id': story_id,
        'total_queries_story': total_queries_story,
        'federated_queries_story': federated_queries_story,
        'federated_bonus': federated_bonus
    })


df_federation = pd.DataFrame(federation_metrics)
df_federation.to_csv('outputs/sparql_federation.csv')

In [6]:
df_proficiency = pd.read_csv("outputs/sparql_proficiency_stage_one.csv", index_col=0)
df_federation = pd.read_csv("outputs/sparql_federation.csv", index_col=0)

df_merged = df_proficiency.merge(df_federation['federated_bonus'], left_index=True, right_index=True)

df_merged.to_csv("outputs/sparql_proficiency_stage_two.csv")

## SPARQL Features Complexity and Diversity

In [8]:
# weights based on usage frequency

sparql_features = ["FILTER", "OPTIONAL", "BIND", "UNION", "MINUS", "COUNT", 
                   "DISTINCT", "LIMIT", "OFFSET", "ORDER BY", "GROUP BY", "SUM", "SAMPLE", 
                   "GROUP_CONCAT", "VALUES", "STR", "LANG", "IF", "CONTAINS", "REPLACE", "ROUND", "ASC", "DESC"]


def count_sparql_features(query, features):
    """
    Count the occurrences of each feature in the query using regex word boundaries.
    Returns a dictionary with the counts.
    """
    feature_counts = {feature: len(re.findall(r'\b' + feature + r'\b', query, re.IGNORECASE)) 
                      for feature in features}
    return feature_counts

def clean_query_text(query):
    # Remove comments
    query = re.sub(r'(^|\s)#.*?(\r\n|\n)', r'\1', query)
    # Normalize whitespace
    query = re.sub(r'\s+', ' ', query)
    # Remove variables like ?varName
    query = re.sub(r'\?[^\s\)\(]*', '', query)
    # Remove URIs like <http://example.org/prop>
    query = re.sub(r'<[^>]*>', '', query)
    return query.strip()


# global features usage
global_feature_counts = {feature: 0 for feature in sparql_features}
total_feature_instances = 0

for story_data in data.values():
    queries = story_data.get('queries', [])
    for query_dict in queries:
        query = query_dict.get('query', '')
        query = clean_query_text(query)
        # Count features for this query.
        feature_counts = count_sparql_features(query, sparql_features)
        for feature, count in feature_counts.items():
            global_feature_counts[feature] += count
            total_feature_instances += count

feature_percentages = {}
# percentage for each feature
if total_feature_instances > 0:
    for feature in sparql_features:
        count = global_feature_counts[feature]
        percentage = count / total_feature_instances
        feature_percentages[feature] = percentage
else:
    for feature in sparql_features:
        feature_percentages[feature] = 0.0

# inverted weights
feature_weights = {}
for feature in sparql_features:
    weight = 1 - feature_percentages[feature]
    feature_weights[feature] = weight

# normalise
min_w = min(feature_weights.values())
max_w = max(feature_weights.values())
if max_w > min_w:
    for feature in sparql_features:
        weight = feature_weights[feature]
        normalised = (weight - min_w) / (max_w - min_w)
        feature_weights[feature] = normalised

story_feature_metrics = {}
for story_id, story_data in data.items():
    queries = story_data.get('queries', [])
    total_queries = len(queries)
    total_feature_counts = {feature: 0 for feature in sparql_features}
    diversity_sum = 0.0
    
    for query_dict in queries:
        query = query_dict.get('query', '')
        query = clean_query_text(query)
        # Count features for this query.
        feature_counts = count_sparql_features(query, sparql_features)
        for feature, count in feature_counts.items():
            total_feature_counts[feature] += count

        # Calculate diversity for this query
        total_features_in_query = sum(feature_counts.values())
        unique_features_in_query = sum(1 for count in feature_counts.values() if count > 0)
        diversity_score = unique_features_in_query / len(sparql_features)

        diversity_sum += diversity_score
        
    weighted_sum = sum(total_feature_counts[feature] * feature_weights.get(feature, 1) for feature in sparql_features)
    # Compute the simple average metric: total features divided by number of queries.
    feature_metric = weighted_sum / total_queries if total_queries > 0 else 0.0
    
    # Calculate average diversity metric
    diversity_metric = diversity_sum / total_queries if total_queries > 0 else 0.0
    
    # Store the metrics for the story using its story_id.
    story_feature_metrics[story_id] = {
        'total_queries': total_queries,
        'weighted_feature_sum': weighted_sum,
        'sparql_feature_metric': feature_metric,
        'diversity_metric': diversity_metric
    }

# Convert the metrics dictionary into a DataFrame.
df_features = pd.DataFrame.from_dict(story_feature_metrics, orient='index')
df_features.index.name = 'story_id'
df_features.reset_index(inplace=True)

# Normalize the sparql_feature_metric using min-max normalization
min_metric = df_features['sparql_feature_metric'].min()
max_metric = df_features['sparql_feature_metric'].max()

if max_metric > min_metric:
    df_features['normalized_sparql_feature_metric'] = (
        (df_features['sparql_feature_metric'] - min_metric) / (max_metric - min_metric)
    )
else:
    df_features['normalized_sparql_feature_metric'] = 0

df_features.to_csv('outputs/sparql_features_scores.csv')

In [10]:
df_proficiency = pd.read_csv("outputs/sparql_proficiency_stage_two.csv", index_col=0)
df_features = pd.read_csv("outputs/sparql_features_scores.csv", index_col=0)

df_merged = df_proficiency.merge(df_features[['normalized_sparql_feature_metric', 'diversity_metric']], left_index=True, right_index=True)
df_merged.to_csv("outputs/sparql_proficiency_stage_three.csv")

## Final Proficiency Values

In [12]:
def compute_sparql_proficiency(row):

    weight_wc = 0.1      # correct queries, we already know that most of them works
    weight_fed = 0.1     # shows advanced usage with federated queries
    weight_feat = 0.4    # features usage reflects query complexity and versatility
    weigh_diversity = 0.4
    
    proficiency = (row['correct_normalized'] * weight_wc +
                   row['federated_bonus'] * weight_fed +
                   row['normalized_sparql_feature_metric'] * weight_feat +
                   row['diversity_metric'] * weigh_diversity)
    
    return proficiency

df_proficiency_final = pd.read_csv("outputs/sparql_proficiency_stage_three.csv", index_col=0)
df_proficiency_final['sparql_proficiency'] = df_proficiency_final.apply(compute_sparql_proficiency, axis=1)

df_proficiency_final.to_csv("outputs/sparql_proficiency_final.csv")